In [1]:
import os

os.chdir("../")
%pwd

'c:\\Users\\jlrodriguez.andreu\\dev\\lab-mlops\\mlops-ml-project'

In [2]:
# aqui crea unas variables d eentorno para mlflow por dagshub
# MLFLOW_TRACKING_URI
# MLFLOW_TRACKING_USERNAME
# MLFLOW_TRACKING_PASSWORD

In [3]:
# entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [4]:
# configuration manager

from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="test"
        )

        return model_evaluation_config










In [6]:
# component
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [7]:
class ModelEvaluation:
    def __init__(self, config=ModelEvaluationConfig):
        self.config=config
    

    def eval_metrics(self, actual, pred):

        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_quantities = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_quantities)

            # saving metrics as local
            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("mae", mae)

            if tracking_url_type_store != "file":

                # register the model
                # there are other ways to use the model registry, with depends on th euse case
                # please refer to the doc for more info
                # https://mlfloe.org/docs/latest/model-registry-.html#api-workflow

                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [8]:
# pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-06-24 12:22:42,104: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-24 12:22:42,108: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-24 12:22:42,111: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-24 12:22:42,112: INFO: common: created directory at: artifacts]
[2024-06-24 12:22:42,113: INFO: common: created directory at: artifacts/model_evaluation]
[2024-06-24 12:22:42,534: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
